In [1]:
import os
import numpy as np
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from tqdm import tqdm
import warnings
from IPython.display import display, HTML, Markdown
import re
import time
import pandas as pd
import json
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, matthews_corrcoef
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=UserWarning, module="neo4j")
import logging
load_dotenv()

True

# Initialization Embeddings model

In [2]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Connect to Neo4j Database

In [3]:
NEO4j_URI = 'bolt://localhost:7687'
NEO4j_USER = 'neo4j'
NEO4j_PASSWORD = os.environ.get('pass')
graph = Neo4jGraph(url=NEO4j_URI, username=NEO4j_USER, password=NEO4j_PASSWORD)

# Hybrid Searching (Semantic + graph)

In [4]:
def retrieve_similar_events(query_text, graph, embeddings, top_k=5, embedding_weight=0.6, structure_weight=0.4, similarity_threshold=0.5):
    """
    Retrieve similar events using a hybrid approach with optimized query performance.
    """
    query_embedding = embeddings.embed_query(query_text)
    
    cypher_query = """
    MATCH (e:Event)
    WHERE e.embedding IS NOT NULL AND e.text IS NOT NULL
    
    // Efficient connection counting using COUNT
    OPTIONAL MATCH (e)-[:RESULTS_IN]->(effect:Effect)
    WITH e, COUNT(effect) AS effect_count
    
    OPTIONAL MATCH (e)<-[:CAUSES]-(cause:Cause)
    WITH e, effect_count, COUNT(cause) AS cause_count
    
    OPTIONAL MATCH (e)-[:HAS_TRIGGER]->(trigger:Trigger)
    WITH e, effect_count, cause_count, COUNT(trigger) AS trigger_count
    
    // Calculate structural score (maintain original binary approach)
    WITH e, effect_count, cause_count, trigger_count,
         CASE 
             WHEN effect_count + cause_count + trigger_count = 0 
             THEN 0.0 
             ELSE 1.0 
         END AS structural_score
    
    // Calculate similarity and hybrid score
    WITH e, effect_count, cause_count, trigger_count, structural_score,
         gds.similarity.cosine(e.embedding, $query_embedding) AS embedding_similarity,
         ($embedding_weight * gds.similarity.cosine(e.embedding, $query_embedding) + $structure_weight * structural_score) AS hybrid_score
    
    WHERE hybrid_score >= $similarity_threshold
    
    // Late text collection for final candidates
    CALL {
        WITH e
        OPTIONAL MATCH (e)-[:RESULTS_IN]->(effect:Effect)
        RETURN COLLECT(effect.text) AS effect_texts
    }
    CALL {
        WITH e
        OPTIONAL MATCH (e)<-[:CAUSES]-(cause:Cause)
        RETURN COLLECT(cause.text) AS cause_texts
    }
    CALL {
        WITH e
        OPTIONAL MATCH (e)-[:HAS_TRIGGER]->(trigger:Trigger)
        RETURN COLLECT(trigger.text) AS trigger_texts
    }
    
    RETURN e.text AS text,
           e.id AS event_id,
           hybrid_score,
           embedding_similarity,
           structural_score,
           effect_count,
           cause_count,
           trigger_count,
           effect_texts,
           cause_texts,
           trigger_texts
    ORDER BY hybrid_score DESC
    LIMIT $top_k
    """
    
    try:
        logging.getLogger("neo4j").setLevel(logging.ERROR)
        results = graph.query(
            cypher_query,
            {
                "query_embedding": query_embedding,
                "embedding_weight": float(embedding_weight),
                "structure_weight": float(structure_weight),
                "similarity_threshold": float(similarity_threshold),
                "top_k": int(top_k)
            }
        )
        
        similar_events = []
        for record in results:
            if record['text']:
                similar_events.append({
                    'text': record['text'],
                    'event_id': record.get('event_id', 'N/A'),
                    'hybrid_score': round(float(record['hybrid_score']), 3),
                    'embedding_similarity': round(float(record['embedding_similarity']), 3),
                    'structural_score': round(float(record['structural_score']), 3),
                    'connection_counts': {
                        'effects': int(record['effect_count']),
                        'causes': int(record['cause_count']),
                        'triggers': int(record['trigger_count'])
                    },
                    'connected_nodes': {
                        'causes': [str(cause) for cause in record['cause_texts'] if cause],
                        'effects': [str(effect) for effect in record['effect_texts'] if effect],
                        'triggers': [str(trigger) for trigger in record['trigger_texts'] if trigger]
                    }
                })
        
        return similar_events
    
    except Exception as e:
        print(f"Error during query execution: {str(e)}")
        return []

# Testing out Hybrid Search query 

In [23]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
query_example = "I observed the attack on the police , I have no doubt about it , '' Modiba said during cross-examination ."
similar_events = retrieve_similar_events(
    query_example,
    graph,
    embeddings,
    top_k=25,
    embedding_weight=0.6,
    structure_weight=0.4,
    similarity_threshold=0.5
)

print(f"\nFound {len(similar_events)} similar events:")
for i, event in enumerate(similar_events, 1):
    print(f"\n{i}. Event: {event['text']}")
    print(f"   ID: {event['event_id']}")
    print(f"   Hybrid Score: {event['hybrid_score']}")
    print(f"   Embedding Similarity: {event['embedding_similarity']}")
    print(f"   Structural Score: {event['structural_score']}")
    
    # Print connected nodes in the requested format
    print("   Connected Nodes:")
    print("   Causes:")
    for cause in event['connected_nodes']['causes']:
        print(f"   - {cause}")
    print("   Effects:")
    for effect in event['connected_nodes']['effects']:
        print(f"   - {effect}")
    print("   Triggers:")
    for trigger in event['connected_nodes']['triggers']:
        print(f"   - {trigger}")
    print("   ---")


Found 25 similar events:

1. Event: i observed the attack on the police , i have no doubt about it , '' modiba said during cross-examination .
   ID: event_20
   Hybrid Score: 1.0
   Embedding Similarity: 1.0
   Structural Score: 1.0
   Connected Nodes:
   Causes:
   - Modiba said
   Effects:
   - that he observed the attack on the police and had no doubt about it
   Triggers:
   - during cross-examination
   ---

2. Event: thanks to the providential arrest of a terrorist in mumbai and his subsequent interrogation , the police are asserting with a considerable degree of confidence that the let planned and orchestrated the attacks that took the lives of more than 180 people .
   ID: event_1131
   Hybrid Score: 0.757
   Embedding Similarity: 0.595
   Structural Score: 1.0
   Connected Nodes:
   Causes:
   - Thanks to the providential arrest of a terrorist in Mumbai and his subsequent interrogation
   Effects:
   - the police are asserting with a considerable degree of confidence that th

# XML Prompting and LLM Generation

In [28]:
def generate(llm, input, graph, embeddings, similar_number):
    prompt="""
    <task>
        You are an advanced language model tasked with identifying and tagging explicit cause-and-effect relationships in complex sentences. 
        <rule>Do not modify the original text when adding tags.</rule>
    </task>

    <VERY IMPORTANT RULE>Do not give output unless all tags are present in sentence (All tags meaning cause, effect, and trigger should be present). If you miss this in the end the answer will be considered wrong</VERY IMPORTANT RULE>

    <steps>
        <step number="1">
            <title>Causality Determination</title>
            <description>
                <point>Analyze the input sentence to determine if it contains an explicit causal relationship.</point>
                <point>If the sentence is <b>not causal</b>, output <b>NoTag</b> and set the label to <b>0</b>. <rule>Do not perform any tagging.</rule></point>
                <point>If the sentence is causal, proceed only if <b>all three elements</b>—cause, trigger, and effect—are present.</point>
                <point>If any element is missing, output <b>NoTag</b> and set the label to <b>0</b>.</point>
            </description>
        </step>

        <step number="2">
            <title>Tagging Instructions</title>
            <tags>
                <tag>
                    <name>Cause</name>
                    <format>&lt;cause&gt;...&lt;/cause&gt;</format>
                    <definition>The event, action, or condition that leads to an outcome.</definition>
                    <question>What caused this?</question>
                </tag>
                <tag>
                    <name>Trigger</name>
                    <format>&lt;trigger&gt;...&lt;/trigger&gt;</format>
                    <definition>The word or phrase that explicitly indicates causality (e.g., because, due to).</definition>
                </tag>
                <tag>
                    <name>Effect</name>
                    <format>&lt;effect&gt;...&lt;/effect&gt;</format>
                    <definition>The outcome or result of the cause.</definition>
                    <question>What is the result of this?</question>
                </tag>
            </tags>
        </step>

        <step number="3">
            <title>Causality Tests</title>
            <description>
                <point>The effect should answer "Why?" (i.e., it must clearly result from the cause).</point>
                <point>The cause must precede the effect in time.</point>
                <point>The effect should not occur without the cause.</point>
                <point>The cause and effect cannot be swapped without changing the meaning.</point>
                <point>The sentence can be rephrased as “X causes Y” or “Due to X, Y.”</point>
            </description>
        </step>

        <step number="4">
            <title>Additional Guidelines</title>
            <guidelines>
                <point>The sentence must have explicit cause-effect wording.</point>
                <point>If there are multiple causes or effects, tag each instance separately.</point>
                <point>Do not alter the original sentence structure when adding tags.</point>
                <point>The final output must include all three tags: <b>&lt;cause&gt;</b>, <b>&lt;trigger&gt;</b>, and <b>&lt;effect&gt;</b>.</point>
                <point>If any tag is missing, consider the sentence non-causal.</point>
                <point>If no explicit causal relationship is identified, output <b>NoTag</b>.</point>
            </guidelines>
        </step>

        <step number="5">
            <title>Output Format</title>
            <output>
                <format>Return a JSON object with two keys: "tagged" and "label".</format>
                <key name="tagged">Contains the original sentence with all tags added.</key>
                <key name="label">Set to <b>1</b> if a causal relationship is successfully tagged; otherwise, <b>0</b>.</key>
            </output>
        </step>
    </steps>

    <examples>
        <example>
            <input>Biswal, who had travelled to the city from Washington to speak at the Indian Consulate General's Media-India Lecture Series yesterday, said not only were there a large number of Indian victims in the attack but there were Americans also who lost their lives.</input>
            <tagged>Biswal, who had <trigger>travelled to the city from Washington</trigger> to <cause>speak at the Indian Consulate General's Media-India Lecture Series</cause> yesterday, said <effect>not only were there a large number of Indian victims in the attack</effect> but <effect>there were Americans also who lost their lives</effect>.</tagged>
            <label>1</label>
        </example>

        <more examples>
        {similar_examples}
        </more examples>

        <example>
            <input>The African nationals had sustained minor injuries in the attack that took place in South Delhi's Mehrauli area on Friday.</input>
            <tagged>NoTag</tagged>
            <label>0</label>
        </example>
        
        <example>
            <input>On Sunday , Swaraj spoke to Rajnath Singh and Delhi Lieutenant Governor Najeeb Jung about the attacks and said she was assured that the culprits would soon be arrested .</input>
            <tagged>NoTag</tagged>
            <label>0</label>
        </example>

        <example>
            <input>We will continue our strike till we get an assurance from the Government .</input>
            <tagged>NoTag</tagged>
            <label>0</label>
        </example>

        <example>
            <input>There is no information about the involvement of the NDFB(S) underground group in the blast , he added .</input>
            <tagged>NoTag</tagged>
            <label>0</label>
        </example>

    </examples>

    <qa>
        <requirement>For sentences that are tagged as causal, include a brief description explaining the cause-and-effect relationship.</requirement>
        <example>
            <sentence>&lt;effect>The participants raised slogans&lt;/effect> &lt;trigger>demanding&lt;/trigger> &lt;cause>stern action to check atrocities against women in the name of moral policing&lt;/cause></sentence>
            <explanation>Why did they raise slogans? To demand action against atrocities.</explanation>
            <label>1</label>
        </example>
    </qa>

    <final_instruction>
        Now, process the following input text:
        <input>{input_text}</input>
        <note>Reconsider your decisions step by step internally (chain-of-thought) but do not output your reasoning.</note>
        <IMPORTANT>Do not give output unless all tags are present in sentence (All tags meaning cause, effect, and trigger should be present).</IMPORTANT>
    </final_instruction>
    """
    prompt_template = PromptTemplate(
		input_variables=["similar_examples", "input_text"],
		template=prompt
	)

    llm_chain = LLMChain(
    	llm=llm,
    	prompt=prompt_template
	)

    similar_events = retrieve_similar_events(
        input,
        graph,
        embeddings,
        top_k=similar_number,
        embedding_weight=0.6,
        structure_weight=0.4,
        similarity_threshold=0.5
    )
    
    output = llm_chain.run(
	    similar_examples=similar_events,
	    input_text=input
	)

    return output

In [29]:
llm = ChatGroq(
    groq_api_key=os.environ.get('GROQ_API'),
    model_name='deepseek-r1-distill-llama-70b'
)

In [31]:
query_example = "As commuters rushed past , activists waved flags in the sticky night air and erected tall blue and white banners urging them to join the struggle for self-determination ."
llm_output = generate(llm, query_example, graph, embeddings, 5)

print(llm_output)

<think>
Alright, I'm trying to figure out whether the given sentence has an explicit cause-and-effect relationship as per the task's instructions. The sentence is: "As commuters rushed past , activists waved flags in the sticky night air and erected tall blue and white banners urging them to join the struggle for self-determination."

First, I need to determine if there's a causal relationship. The sentence describes two actions: commuters rushing past and activists waving flags and erecting banners. I'm looking for a trigger word that indicates causality, like 'because', 'due to', 'since', etc. But I don't see any such words here. 

Next, I check if there's an implicit cause. The activists' actions (waving flags and erecting banners) might be a response to the commuters, but without a trigger word, it's not explicit. The phrase "as commuters rushed past" could imply that the activists' actions happened while the commuters were moving, but it doesn't clearly state that the commuters' m

# Get predictions with 25 examples retrieved for Test Set

In [17]:
df = pd.read_csv('test_data.csv')

In [18]:
df

,index,text,label,agreement,num_votes,sample_set
0,train_10_0,The movement was catapulted into the headlines...,1,0.666667,6,train_10
1,train_10_1,Several thousand protesters took to the street...,1,0.833333,6,train_10
2,train_10_2,"“ They try every means to oppress us , ” compl...",1,0.833333,6,train_10
3,train_10_3,"Chan Ho-tin , the 25-year-old founder of the H...",0,0.833333,6,train_10
4,train_10_4,“ It is somehow like a revolution – no one wil...,1,0.500000,6,train_10
...,...,...,...,...,...,...
318,train_10_318,"Police spokesman , Lieutenant Colonel Thulani ...",0,0.500000,4,train_10
319,train_10_319,"National Freedom Party spokesman , Sabelo Sigu...",1,0.750000,4,train_10
320,train_10_320,He described the shooting as `` evil '' .,0,1.000000,4,train_10
321,train_10_321,"On Sunday , three explosions in the south of t...",1,1.000000,4,train_10


In [33]:
import threading
import time
import json
import re
from tqdm import tqdm

lock = threading.Lock()
sem = threading.Semaphore(5)

if os.path.exists("test_pred_25.json"):
    with open("test_pred_25.json", "r", encoding="utf-8") as f:
        data = json.load(f)
else:
    data = []
    with open("test_pred_25.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

processed = {item["original"] for item in data}

total_rows = len(df)
start_time = time.time()
pbar = tqdm(total=total_rows, desc="Processing", unit="rows")

def worker(text):
    output = generate(llm, text, graph, embeddings, 25)
    match = re.search(r"\{.*\}", output, re.DOTALL)
    if not match:
        with lock:
            pbar.update(1)
        sem.release()
        return

    try:
        j = json.loads(match.group(0))
    except json.JSONDecodeError:
        with lock:
            pbar.update(1)
        sem.release()
        return

    tagged = j.get("tagged", "")
    label = j.get("label", "")

    with lock:
        data.append({"tagged": tagged, "label": label, "original": text})
        with open("test_pred_25.json", "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        pbar.update(1)
        elapsed = time.time() - start_time
        avg = elapsed / pbar.n
        rem = avg * (total_rows - pbar.n)
        pbar.set_postfix({"ETA (sec)": f"{rem:.2f}"})
        print(f"\nOriginal Sentence: {text}")
        print(f"Tagged Sentence: {tagged}")
        print(f"Label: {label}")
        print("---")
    sem.release()

threads = []
for _, row in df.iterrows():
    text = row["text"]
    if text in processed:
        pbar.update(1)
        continue
    sem.acquire()
    t = threading.Thread(target=worker, args=(text,))
    t.start()
    threads.append(t)

for t in threads:
    t.join()

pbar.close()

Processing:   0%|          | 1/323 [00:02<15:29,  2.89s/rows, ETA (sec)=932.19]


Original Sentence: The movement was catapulted into the headlines in early August when the semi-autonomous city – which returned to Chinese rule almost two decades ago , in 1997 – saw the first pro-independence rally in its history .
Tagged Sentence: The movement was catapulted into the headlines in early August <trigger>when</trigger> the semi-autonomous city – which returned to Chinese rule almost two decades ago, in 1997 – saw <cause>the first pro-independence rally in its history</cause>.
Label: 1
---


Processing:   1%|          | 2/323 [00:05<12:58,  2.43s/rows, ETA (sec)=805.95]


Original Sentence: Chan Ho-tin , the 25-year-old founder of the Hong Kong national party , one of several recently formed pro-independence groups , said he had been surprised how many people attended August ’ s historic rally , which he organised .
Tagged Sentence: NoTag
Label: 0
---


Processing:   1%|          | 3/323 [00:05<08:47,  1.65s/rows, ETA (sec)=610.63]

...

Processing:  99%|█████████▉| 321/323 [06:04<00:01,  1.10rows/s, ETA (sec)=2.27]


Original Sentence: He described the shooting as `` evil '' .
Tagged Sentence: NoTag
Label: 0
---


Processing: 100%|█████████▉| 322/323 [06:05<00:00,  1.26rows/s, ETA (sec)=1.13]


Original Sentence: On Sunday , three explosions in the south of the region killed two people and injured many others .
Tagged Sentence: NoTag
Label: 0
---


Processing: 100%|██████████| 323/323 [06:07<00:00,  1.14s/rows, ETA (sec)=1.13]


# Get F1 Score

In [34]:
with open("test_pred_25.json", "r", encoding="utf-8") as file:
    data = json.load(file)

In [35]:
df_predictions = pd.DataFrame(data, columns=[ "tagged", "label", "original"])

In [36]:
df_predictions

,tagged,label,original
0,The movement was catapulted into the headlines...,1,The movement was catapulted into the headlines...
1,NoTag,0,"Chan Ho-tin , the 25-year-old founder of the H..."
2,Several thousand <effect>protesters took to th...,1,Several thousand protesters took to the street...
3,NoTag,0,“ It is somehow like a revolution – no one wil...
4,"As commuters rushed past , <cause>activists wa...",1,"As commuters rushed past , activists waved fla..."
...,...,...,...
315,NoTag,0,Newly appointed municipal manager of Richmond ...
316,"Police spokesman , Lieutenant Colonel Thulani ...",1,"Police spokesman , Lieutenant Colonel Thulani ..."
317,NoTag,0,"National Freedom Party spokesman , Sabelo Sigu..."
318,NoTag,0,He described the shooting as `` evil '' .


In [37]:
df_predictions.rename(columns={
    "original": "original_sentence",
    "tagged": "tagged_sentence",
    "label": "label"
}, inplace=True)

In [38]:
df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce')

In [39]:
df_predictions

,tagged_sentence,label,original_sentence
0,The movement was catapulted into the headlines...,1,The movement was catapulted into the headlines...
1,NoTag,0,"Chan Ho-tin , the 25-year-old founder of the H..."
2,Several thousand <effect>protesters took to th...,1,Several thousand protesters took to the street...
3,NoTag,0,“ It is somehow like a revolution – no one wil...
4,"As commuters rushed past , <cause>activists wa...",1,"As commuters rushed past , activists waved fla..."
...,...,...,...
315,NoTag,0,Newly appointed municipal manager of Richmond ...
316,"Police spokesman , Lieutenant Colonel Thulani ...",1,"Police spokesman , Lieutenant Colonel Thulani ..."
317,NoTag,0,"National Freedom Party spokesman , Sabelo Sigu..."
318,NoTag,0,He described the shooting as `` evil '' .


In [40]:
df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce').fillna(0).astype(int)

In [41]:
common_df = df.merge(df_predictions, left_on="text", right_on="original_sentence", suffixes=('_df', '_pred'))

In [42]:
common_df

,index,text,label_df,agreement,num_votes,sample_set,tagged_sentence,label_pred,original_sentence
0,train_10_0,The movement was catapulted into the headlines...,1,0.666667,6,train_10,The movement was catapulted into the headlines...,1,The movement was catapulted into the headlines...
1,train_10_1,Several thousand protesters took to the street...,1,0.833333,6,train_10,Several thousand <effect>protesters took to th...,1,Several thousand protesters took to the street...
2,train_10_2,"“ They try every means to oppress us , ” compl...",1,0.833333,6,train_10,"“ They try every means to oppress us , ” compl...",1,"“ They try every means to oppress us , ” compl..."
3,train_10_3,"Chan Ho-tin , the 25-year-old founder of the H...",0,0.833333,6,train_10,NoTag,0,"Chan Ho-tin , the 25-year-old founder of the H..."
4,train_10_4,“ It is somehow like a revolution – no one wil...,1,0.500000,6,train_10,NoTag,0,“ It is somehow like a revolution – no one wil...
...,...,...,...,...,...,...,...,...,...
315,train_10_317,Newly appointed municipal manager of Richmond ...,0,1.000000,4,train_10,NoTag,0,Newly appointed municipal manager of Richmond ...
316,train_10_318,"Police spokesman , Lieutenant Colonel Thulani ...",0,0.500000,4,train_10,"Police spokesman , Lieutenant Colonel Thulani ...",1,"Police spokesman , Lieutenant Colonel Thulani ..."
317,train_10_319,"National Freedom Party spokesman , Sabelo Sigu...",1,0.750000,4,train_10,NoTag,0,"National Freedom Party spokesman , Sabelo Sigu..."
318,train_10_320,He described the shooting as `` evil '' .,0,1.000000,4,train_10,NoTag,0,He described the shooting as `` evil '' .


In [43]:
f1 = f1_score(common_df['label_df'], common_df['label_pred'])
print("F1 Score:", f1)

accuracy = accuracy_score(common_df['label_df'], common_df['label_pred'])
print("Accuracy:", accuracy)

precision = precision_score(common_df['label_df'], common_df['label_pred'])
print("Precision:", precision)

recall = recall_score(common_df['label_df'], common_df['label_pred'])
print("Recall:", recall)

MCC = matthews_corrcoef(common_df['label_df'], common_df['label_pred'])
print("Matthews Correlation Coefficient:", MCC)

F1 Score: 0.7979002624671916
Accuracy: 0.759375
Precision: 0.7414634146341463
Recall: 0.8636363636363636
Matthews Correlation Coefficient: 0.513837988318737


# Now for 30 Examples

In [49]:
import threading
import time
import json
import re
from tqdm import tqdm

lock = threading.Lock()
sem = threading.Semaphore(5)

if os.path.exists("test_pred_30.json"):
    with open("test_pred_30.json", "r", encoding="utf-8") as f:
        data = json.load(f)
else:
    data = []
    with open("test_pred_30.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

processed = {item["original"] for item in data}

total_rows = len(df)
start_time = time.time()
pbar = tqdm(total=total_rows, desc="Processing", unit="rows")

def worker(text):
    output = generate(llm, text, graph, embeddings, 30)
    match = re.search(r"\{.*\}", output, re.DOTALL)
    if not match:
        with lock:
            pbar.update(1)
        sem.release()
        return

    try:
        j = json.loads(match.group(0))
    except json.JSONDecodeError:
        with lock:
            pbar.update(1)
        sem.release()
        return

    tagged = j.get("tagged", "")
    label = j.get("label", "")

    with lock:
        data.append({"tagged": tagged, "label": label, "original": text})
        with open("test_pred_30.json", "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        pbar.update(1)
        elapsed = time.time() - start_time
        avg = elapsed / pbar.n
        rem = avg * (total_rows - pbar.n)
        pbar.set_postfix({"ETA (sec)": f"{rem:.2f}"})
        print(f"\nOriginal Sentence: {text}")
        print(f"Tagged Sentence: {tagged}")
        print(f"Label: {label}")
        print("---")
    sem.release()

threads = []
for _, row in df.iterrows():
    text = row["text"]
    if text in processed:
        pbar.update(1)
        continue
    sem.acquire()
    t = threading.Thread(target=worker, args=(text,))
    t.start()
    threads.append(t)

for t in threads:
    t.join()

pbar.close()

Processing:   1%|          | 3/323 [00:02<04:06,  1.30rows/s, ETA (sec)=311.43]


Original Sentence: “ It is somehow like a revolution – no one will have a plan for a revolution. ” On a recent evening , Edward Leung and a group of fellow activists gathered outside a metro station in Hong Kong ’ s New Territories to lend their support to a Youngspiration campaign event .
Tagged Sentence: NoTag
Label: 0
---

Original Sentence: Several thousand protesters took to the streets after six pro-independence candidates , foremost amongst them Leung , were barred from the upcoming election in what critics condemned as an act of political censorship designed to snuff out opposition to Beijing ’ s authoritarian rule .
Tagged Sentence: Several thousand protesters <trigger>after</trigger> <cause>six pro-independence candidates, foremost amongst them Leung, were barred from the upcoming election</cause> took to the streets <effect>in what critics condemned as an act of political censorship designed to snuff out opposition to Beijing’s authoritarian rule</effect>.
Label: 1
---

Ori

Processing:   2%|▏         | 5/323 [00:03<03:08,  1.69rows/s, ETA (sec)=252.37]


Original Sentence: “ They try every means to oppress us , ” complained Leung , one of the leaders of Hong Kong Indigenous , a so-called “ localist ” political group founded in the wake of 2014 ’ s umbrella movement protests to combat what its members see as China ’ s erosion of the city ’ s way of life .
Tagged Sentence: “They try every means to oppress us,” <effect>complained Leung, one of the leaders of Hong Kong Indigenous, a so-called “localist” political group</effect> <trigger>founded in the wake of</trigger> <cause>2014’s umbrella movement protests to combat what its members see as China’s erosion of the city’s way of life</cause>.
Label: 1
---

Original Sentence: Chan Ho-tin , the 25-year-old founder of the Hong Kong national party , one of several recently formed pro-independence groups , said he had been surprised how many people attended August ’ s historic rally , which he organised .
Tagged Sentence: NoTag
Label: 0
---


Processing:   2%|▏         | 7/323 [00:05<03:31,  1.50rows/s, ETA (sec)=256.04]

...


Original Sentence: National Freedom Party spokesman , Sabelo Sigudu , said : `` The National Freedom Party ( NFP ) wishes to express words of condolence on the sad shooting and killing of Mr Sithole , the Municipality Manager of Richmond Local Municipality , which took place this morning . ''
Tagged Sentence: National Freedom Party spokesman, Sabelo Sigudu, said: `` The National Freedom Party ( NFP ) wishes to <effect>express words of condolence</effect> <trigger>on</trigger> the <cause>sad shooting and killing of Mr Sithole, the Municipality Manager of Richmond Local Municipality, which took place this morning</cause>. ''
Label: 1
---


Processing:  99%|█████████▉| 319/323 [06:33<00:05,  1.43s/rows, ETA (sec)=4.93]


Original Sentence: On Sunday , three explosions in the south of the region killed two people and injured many others .
Tagged Sentence: On Sunday , three <cause>explosions in the south of the region</cause> <trigger>killed</trigger> <effect>two people</effect> and <trigger>injured</trigger> <effect>many others</effect> .
Label: 1
---


Processing:  99%|█████████▉| 320/323 [06:34<00:03,  1.23s/rows, ETA (sec)=3.70]


Original Sentence: Uighur groups abroad say the clashes are a desperate protest against religious oppression and economic marginalisation .
Tagged Sentence: Uighur groups abroad say the <effect>clashes</effect> are a desperate <trigger>protest against</trigger> <cause>religious oppression and economic marginalisation</cause>.
Label: 1
---


# Get F1 Score

In [50]:
with open("test_pred_30.json", "r", encoding="utf-8") as file:
    data = json.load(file)

In [51]:
df_predictions = pd.DataFrame(data, columns=[ "tagged", "label", "original"])

In [52]:
df_predictions

,tagged,label,original
0,NoTag,0,“ It is somehow like a revolution – no one wil...
1,Several thousand protesters <trigger>after</tr...,1,Several thousand protesters took to the street...
2,The movement was <effect>catapulted into the h...,1,The movement was catapulted into the headlines...
3,"“They try every means to oppress us,” <effect>...",1,"“ They try every means to oppress us , ” compl..."
4,NoTag,0,"Chan Ho-tin , the 25-year-old founder of the H..."
...,...,...,...
311,NoTag,0,Newly appointed municipal manager of Richmond ...
312,NoTag,0,He described the shooting as `` evil '' .
313,"National Freedom Party spokesman, Sabelo Sigud...",1,"National Freedom Party spokesman , Sabelo Sigu..."
314,"On Sunday , three <cause>explosions in the sou...",1,"On Sunday , three explosions in the south of t..."


In [53]:
df_predictions.rename(columns={
    "original": "original_sentence",
    "tagged": "tagged_sentence",
    "label": "label"
}, inplace=True)

In [54]:
df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce')

In [55]:
df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce').fillna(0).astype(int)

In [56]:
common_df = df.merge(df_predictions, left_on="text", right_on="original_sentence", suffixes=('_df', '_pred'))

In [57]:
common_df

,index,text,label_df,agreement,num_votes,sample_set,tagged_sentence,label_pred,original_sentence
0,train_10_0,The movement was catapulted into the headlines...,1,0.666667,6,train_10,The movement was <effect>catapulted into the h...,1,The movement was catapulted into the headlines...
1,train_10_1,Several thousand protesters took to the street...,1,0.833333,6,train_10,Several thousand protesters <trigger>after</tr...,1,Several thousand protesters took to the street...
2,train_10_2,"“ They try every means to oppress us , ” compl...",1,0.833333,6,train_10,"“They try every means to oppress us,” <effect>...",1,"“ They try every means to oppress us , ” compl..."
3,train_10_3,"Chan Ho-tin , the 25-year-old founder of the H...",0,0.833333,6,train_10,NoTag,0,"Chan Ho-tin , the 25-year-old founder of the H..."
4,train_10_4,“ It is somehow like a revolution – no one wil...,1,0.500000,6,train_10,NoTag,0,“ It is somehow like a revolution – no one wil...
...,...,...,...,...,...,...,...,...,...
311,train_10_318,"Police spokesman , Lieutenant Colonel Thulani ...",0,0.500000,4,train_10,NoTag,0,"Police spokesman , Lieutenant Colonel Thulani ..."
312,train_10_319,"National Freedom Party spokesman , Sabelo Sigu...",1,0.750000,4,train_10,"National Freedom Party spokesman, Sabelo Sigud...",1,"National Freedom Party spokesman , Sabelo Sigu..."
313,train_10_320,He described the shooting as `` evil '' .,0,1.000000,4,train_10,NoTag,0,He described the shooting as `` evil '' .
314,train_10_321,"On Sunday , three explosions in the south of t...",1,1.000000,4,train_10,"On Sunday , three <cause>explosions in the sou...",1,"On Sunday , three explosions in the south of t..."


In [58]:
f1 = f1_score(common_df['label_df'], common_df['label_pred'])
print("F1 Score:", f1)

accuracy = accuracy_score(common_df['label_df'], common_df['label_pred'])
print("Accuracy:", accuracy)

precision = precision_score(common_df['label_df'], common_df['label_pred'])
print("Precision:", precision)

recall = recall_score(common_df['label_df'], common_df['label_pred'])
print("Recall:", recall)

MCC = matthews_corrcoef(common_df['label_df'], common_df['label_pred'])
print("Matthews Correlation Coefficient:", MCC)

F1 Score: 0.8119891008174387
Accuracy: 0.7816455696202531
Precision: 0.772020725388601
Recall: 0.8563218390804598
Matthews Correlation Coefficient: 0.5575041404023712


# 40 Examples

In [59]:
import threading
import time
import json
import re
from tqdm import tqdm

lock = threading.Lock()
sem = threading.Semaphore(5)

if os.path.exists("test_pred_40.json"):
    with open("test_pred_40.json", "r", encoding="utf-8") as f:
        data = json.load(f)
else:
    data = []
    with open("test_pred_40.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

processed = {item["original"] for item in data}

total_rows = len(df)
start_time = time.time()
pbar = tqdm(total=total_rows, desc="Processing", unit="rows")

def worker(text):
    output = generate(llm, text, graph, embeddings, 40)
    match = re.search(r"\{.*\}", output, re.DOTALL)
    if not match:
        with lock:
            pbar.update(1)
        sem.release()
        return

    try:
        j = json.loads(match.group(0))
    except json.JSONDecodeError:
        with lock:
            pbar.update(1)
        sem.release()
        return

    tagged = j.get("tagged", "")
    label = j.get("label", "")

    with lock:
        data.append({"tagged": tagged, "label": label, "original": text})
        with open("test_pred_40.json", "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        pbar.update(1)
        elapsed = time.time() - start_time
        avg = elapsed / pbar.n
        rem = avg * (total_rows - pbar.n)
        pbar.set_postfix({"ETA (sec)": f"{rem:.2f}"})
        print(f"\nOriginal Sentence: {text}")
        print(f"Tagged Sentence: {tagged}")
        print(f"Label: {label}")
        print("---")
    sem.release()

threads = []
for _, row in df.iterrows():
    text = row["text"]
    if text in processed:
        pbar.update(1)
        continue
    sem.acquire()
    t = threading.Thread(target=worker, args=(text,))
    t.start()
    threads.append(t)

for t in threads:
    t.join()

pbar.close()

Processing:   0%|          | 1/323 [00:02<13:46,  2.57s/rows, ETA (sec)=828.49]


Original Sentence: Chan Ho-tin , the 25-year-old founder of the Hong Kong national party , one of several recently formed pro-independence groups , said he had been surprised how many people attended August ’ s historic rally , which he organised .
Tagged Sentence: NoTag
Label: 0
---


Processing:   1%|          | 4/323 [00:03<03:56,  1.35rows/s, ETA (sec)=245.16]


Original Sentence: Several thousand protesters took to the streets after six pro-independence candidates , foremost amongst them Leung , were barred from the upcoming election in what critics condemned as an act of political censorship designed to snuff out opposition to Beijing ’ s authoritarian rule .
Tagged Sentence: Several thousand <effect>protesters took to the streets</effect> <trigger>after</trigger> six pro-independence candidates, foremost amongst them Leung, were barred from the upcoming election in what critics condemned as an act of political censorship designed to snuff out opposition to Beijing’s authoritarian rule.
Label: 1
---

Original Sentence: The movement was catapulted into the headlines in early August when the semi-autonomous city – which returned to Chinese rule almost two decades ago , in 1997 – saw the first pro-independence rally in its history .
Tagged Sentence: The movement was catapulted into the headlines in early August <trigger>when</trigger> the semi

Processing:   2%|▏         | 5/323 [00:03<01:53,  2.80rows/s, ETA (sec)=201.42]

...


Original Sentence: He described the shooting as `` evil '' .
Tagged Sentence: NoTag
Label: 0
---


Processing:  99%|█████████▉| 319/323 [07:16<00:05,  1.48s/rows, ETA (sec)=5.48]


Original Sentence: On Sunday , three explosions in the south of the region killed two people and injured many others .
Tagged Sentence: On Sunday , <trigger>killed two people and injured many others</trigger> <cause>three explosions in the south of the region</cause> .
Label: 1
---


Processing:  99%|█████████▉| 320/323 [07:17<00:04,  1.37s/rows, ETA (sec)=4.10]


Original Sentence: Uighur groups abroad say the clashes are a desperate protest against religious oppression and economic marginalisation .
Tagged Sentence: Uighur groups abroad say the <effect>clashes</effect> are a desperate <trigger>protest</trigger> <cause>against religious oppression and economic marginalisation</cause>.
Label: 1
---


# Get F1 Score

In [62]:
with open("test_pred_40.json", "r", encoding="utf-8") as file:
    data = json.load(file)

In [63]:
df_predictions = pd.DataFrame(data, columns=[ "tagged", "label", "original"])

In [64]:
df_predictions.rename(columns={
    "original": "original_sentence",
    "tagged": "tagged_sentence",
    "label": "label"
}, inplace=True)

In [65]:
df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce')

In [66]:
df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce').fillna(0).astype(int)

In [67]:
common_df = df.merge(df_predictions, left_on="text", right_on="original_sentence", suffixes=('_df', '_pred'))

In [68]:
f1 = f1_score(common_df['label_df'], common_df['label_pred'])
print("F1 Score:", f1)

accuracy = accuracy_score(common_df['label_df'], common_df['label_pred'])
print("Accuracy:", accuracy)

precision = precision_score(common_df['label_df'], common_df['label_pred'])
print("Precision:", precision)

recall = recall_score(common_df['label_df'], common_df['label_pred'])
print("Recall:", recall)

MCC = matthews_corrcoef(common_df['label_df'], common_df['label_pred'])
print("Matthews Correlation Coefficient:", MCC)

F1 Score: 0.8288770053475936
Accuracy: 0.8
Precision: 0.7868020304568528
Recall: 0.8757062146892656
Matthews Correlation Coefficient: 0.594828691904356


# 50 examples

In [69]:
import threading
import time
import json
import re
from tqdm import tqdm

lock = threading.Lock()
sem = threading.Semaphore(5)

if os.path.exists("test_pred_50.json"):
    with open("test_pred_50.json", "r", encoding="utf-8") as f:
        data = json.load(f)
else:
    data = []
    with open("test_pred_50.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

processed = {item["original"] for item in data}

total_rows = len(df)
start_time = time.time()
pbar = tqdm(total=total_rows, desc="Processing", unit="rows")

def worker(text):
    output = generate(llm, text, graph, embeddings, 50)
    match = re.search(r"\{.*\}", output, re.DOTALL)
    if not match:
        with lock:
            pbar.update(1)
        sem.release()
        return

    try:
        j = json.loads(match.group(0))
    except json.JSONDecodeError:
        with lock:
            pbar.update(1)
        sem.release()
        return

    tagged = j.get("tagged", "")
    label = j.get("label", "")

    with lock:
        data.append({"tagged": tagged, "label": label, "original": text})
        with open("test_pred_50.json", "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        pbar.update(1)
        elapsed = time.time() - start_time
        avg = elapsed / pbar.n
        rem = avg * (total_rows - pbar.n)
        pbar.set_postfix({"ETA (sec)": f"{rem:.2f}"})
        print(f"\nOriginal Sentence: {text}")
        print(f"Tagged Sentence: {tagged}")
        print(f"Label: {label}")
        print("---")
    sem.release()

threads = []
for _, row in df.iterrows():
    text = row["text"]
    if text in processed:
        pbar.update(1)
        continue
    sem.acquire()
    t = threading.Thread(target=worker, args=(text,))
    t.start()
    threads.append(t)

for t in threads:
    t.join()

pbar.close()

Processing:   0%|          | 1/323 [00:02<14:20,  2.67s/rows, ETA (sec)=861.27]


Original Sentence: Chan Ho-tin , the 25-year-old founder of the Hong Kong national party , one of several recently formed pro-independence groups , said he had been surprised how many people attended August ’ s historic rally , which he organised .
Tagged Sentence: NoTag
Label: 0
---


Processing:   1%|          | 3/323 [00:03<04:31,  1.18rows/s, ETA (sec)=361.87]


Original Sentence: Several thousand protesters took to the streets after six pro-independence candidates , foremost amongst them Leung , were barred from the upcoming election in what critics condemned as an act of political censorship designed to snuff out opposition to Beijing ’ s authoritarian rule .
Tagged Sentence: Several thousand protesters took to the streets after <trigger>after</trigger> six pro-independence candidates, foremost amongst them Leung, were barred from the upcoming election in what critics condemned as an act of political censorship designed to snuff out opposition to Beijing’s authoritarian rule.
Label: 1
---

Original Sentence: “ It is somehow like a revolution – no one will have a plan for a revolution. ” On a recent evening , Edward Leung and a group of fellow activists gathered outside a metro station in Hong Kong ’ s New Territories to lend their support to a Youngspiration campaign event .
Tagged Sentence: NoTag
Label: 0
---


Processing:   2%|▏         | 5/323 [00:03<02:16,  2.33rows/s, ETA (sec)=229.33]

...


Original Sentence: He described the shooting as `` evil '' .
Tagged Sentence: NoTag
Label: 0
---


Processing:  99%|█████████▉| 319/323 [08:09<00:08,  2.18s/rows, ETA (sec)=6.14]


Original Sentence: National Freedom Party spokesman , Sabelo Sigudu , said : `` The National Freedom Party ( NFP ) wishes to express words of condolence on the sad shooting and killing of Mr Sithole , the Municipality Manager of Richmond Local Municipality , which took place this morning . ''
Tagged Sentence: National Freedom Party spokesman, Sabelo Sigudu, said: '<cause>the sad shooting and killing of Mr. Sithole, the Municipality Manager of Richmond Local Municipality, which took place this morning</cause>' <trigger>on</trigger> '<effect>the National Freedom Party (NFP) wishes to express words of condolence</effect>'
Label: 1
---

Original Sentence: On Sunday , three explosions in the south of the region killed two people and injured many others .
Tagged Sentence: On Sunday , <trigger>three explosions in the south of the region</trigger> <cause>killed two people and injured many others</cause>.
Label: 1
---


Processing:  99%|█████████▉| 320/323 [08:14<00:04,  1.55s/rows, ETA (sec)=4.64]


Original Sentence: Uighur groups abroad say the clashes are a desperate protest against religious oppression and economic marginalisation .
Tagged Sentence: Uighur groups abroad say the <effect>clashes</effect> are a <cause>desperate protest against religious oppression and economic marginalisation</cause> <trigger>against</trigger>.
Label: 1
---


# Get F1 Score

In [70]:
with open("test_pred_50.json", "r", encoding="utf-8") as file:
    data = json.load(file)

df_predictions = pd.DataFrame(data, columns=[ "tagged", "label", "original"])

df_predictions.rename(columns={
    "original": "original_sentence",
    "tagged": "tagged_sentence",
    "label": "label"
}, inplace=True)

df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce')

df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce').fillna(0).astype(int)

common_df = df.merge(df_predictions, left_on="text", right_on="original_sentence", suffixes=('_df', '_pred'))

f1 = f1_score(common_df['label_df'], common_df['label_pred'])
print("F1 Score:", f1)

accuracy = accuracy_score(common_df['label_df'], common_df['label_pred'])
print("Accuracy:", accuracy)

precision = precision_score(common_df['label_df'], common_df['label_pred'])
print("Precision:", precision)

recall = recall_score(common_df['label_df'], common_df['label_pred'])
print("Recall:", recall)

MCC = matthews_corrcoef(common_df['label_df'], common_df['label_pred'])
print("Matthews Correlation Coefficient:", MCC)

F1 Score: 0.8152173913043478
Accuracy: 0.7854889589905363
Precision: 0.78125
Recall: 0.8522727272727273
Matthews Correlation Coefficient: 0.5637464161838679
